# Database setup

The cell below creates the database and all associated tables. Existing tables will not be cleared. You should-re run this cell if there are changes to the database tables that should be applied.

In [1]:
import pickle
import logging

from itertools import chain
from typing import List, Set, Generator, Dict

from src import settings
from src.database import models
from src.constants import Paths


logger = logging.getLogger(settings.LOGGER_NAME)

# Bind models and create tables
settings.DATABASE.bind(models.MODELS, bind_refs=True, bind_backrefs=True)
settings.DATABASE.create_tables(models.MODELS)

# REST client setup

Clients for Uniprot and Kegg are also created so downloads can be cached during this session and re-used if something goes wrong.

In [2]:
from src.clients import Kegg, UniprotClient

# Use the download caches for UniProt and Kegg clients.
cache_kegg = True
cache_uniprot = True

# Use global clients to cache results in case of error.
kegg_client = Kegg(use_cache=cache_kegg)
uniprot_client = UniprotClient(use_cache=cache_uniprot, verbose=True)

# Annotations

The cell directly below will populate the annotation tables, which includ Pfam, InterPro and Gene Ontology terms. This is optional but if performed will result in less time being required to populate the protein and interaction tables.

In [3]:
from src.parsers import go, interpro, pfam
from src.database.utilities import create_terms
from src.database.models import GeneOntologyTerm, PfamTerm, InterproTerm


with settings.DATABASE.atomic():
    create_terms(
        terms=pfam.parse_clans_file(path=Paths.pfam_clans), 
        model=PfamTerm
    )
    create_terms(
        terms=go.parse_go_obo(path=Paths.go_obo), 
        model=GeneOntologyTerm
    )
    create_terms(
        terms=interpro.parse_entry_list(path=Paths.interpro_entries), 
        model=InterproTerm
    )

[INFO] 2019-09-05 12:43:38,574 go Unzipping '/home/daniel/.tamago/data/go.obo.gz' into '/tmp/uzippedleq4f4ky.obo'


/tmp/uzippedleq4f4ky.obo: fmt(1.2) rel(2019-06-01) 47,444 GO Terms


# Proteins and Interactions

The cell below will parse interactions and update the accessions of all `source` and `target` nodes to the most recent
UniProt accessions. Interactions which cannot be mapped will be discarded.

In [4]:
from src.parsers import kegg, hprd, bioplex, pina, innate
from src.parsers.types import InteractionData
from src.clients.uniprot import UniprotEntry
from src.database.utilities import update_accessions


# Normalizing interactions formats all identifiers/fields and set removes 
# duplicates.
logger.info("Parsing interactions.")
interactions: Set[InteractionData] = set(
    interaction.normalize()
    for interaction in chain(
        kegg.parse_interactions(client=kegg_client),
        hprd.parse_interactions(
            ptms=hprd.parse_ptm(path=Paths.hprd_ptms), 
            xrefs=hprd.parse_xref_mapping(path=Paths.hprd_xref)
        ),
        pina.parse_interactions(path=Paths.pina2_mitab),
        innate.parse_interactions(Paths.innate_all),
        innate.parse_interactions(Paths.innate_curated),
        bioplex.parse_interactions(path=Paths.bioplex),
    )
)

[INFO] 2019-09-05 12:44:34,349 <ipython-input-4-c619f0ce3991> Parsing interactions.
[INFO] 2019-09-05 12:44:36,781 kegg Downloading Kegg to UniProt mapping.
[INFO] 2019-09-05 12:44:36,783 kegg Downloading HSA pathways.
100%|██████████| 335/335 [00:07<00:00, 46.25it/s]
[INFO] 2019-09-05 12:44:47,478 kegg Generating interactions.


In [5]:
# Map all UniProt identifiers to their most recent versions.
logger.info("Fetching updated UniProt identifiers.")
uniprot_ids: Set[str] = set(
    accession
    for interaction in interactions
    for accession in (interaction.source, interaction.target)
)

uniprot_id_mapping: Dict[str, List[str]]
uniprot_id_mapping = uniprot_client.get_mapping_table(
    identifiers=uniprot_ids,
    fr='ACC+ID',
    to="ACC",
    batch_size=500,
)

[INFO] 2019-09-05 12:45:55,433 <ipython-input-5-50afc3efc0e9> Fetching updated UniProt identifiers.
[INFO] 2019-09-05 12:45:55,884 uniprot Requesting 41 batches of size 500.
100%|██████████| 41/41 [03:51<00:00,  5.64s/it]


In [6]:
# Map all source and target nodes to their most recent accessions. This 
# call will also aggregate all equal interactions into a single instance
# after the mapping process.
logger.info("Mapping interactions to updated UniProt identifiers.")
mapped_interactions: List[InteractionData]
mapped_interactions = update_accessions(
    interactions=interactions,
    mapping=uniprot_id_mapping,
)

[INFO] 2019-09-05 12:49:47,310 <ipython-input-6-3a47dc86cc0b> Mapping interactions to updated UniProt identifiers.
[WARNING] 2019-09-05 12:49:48,273 utilities Could not find a mapping for target node 'P170P1'. Skipping interaction {'source': 'Q9BQD3', 'target': 'P170P1', 'labels': [], 'databases': ['innatedb'], 'evidence': [{'pubmed': 'PUBMED:16189514', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-05 12:49:49,633 utilities Could not find a mapping for target node 'P170P1'. Skipping interaction {'source': 'Q9UFG5', 'target': 'P170P1', 'labels': [], 'databases': ['innatedb'], 'evidence': [{'pubmed': 'PUBMED:16189514', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-05 12:49:50,288 utilities Could not find a mapping for target node 'P170P1'. Skipping interaction {'source': 'Q8TAP4', 'target': 'P170P1', 'labels': [], 'databases': ['innatedb'], 'evidence': [{'pubmed': 'PUBMED:16189514', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-05 12:49:51,918 utilities Could not find a mapping for target node 'P32AP1'

# Querying UniProt

The cell below queries UniProt for the XML flat files for each uniprot identifier. This can take a while if this is a first time run, or there are a lot of cache misses.

In [7]:
uniprot_entries: Generator[UniprotEntry, None, None]
uniprot_entries = uniprot_client.get_entries(
    identifiers=uniprot_ids, 
    batch_size=500
)

# Database population

The cell below will parse and validate all interactions, including UniProt identifiers and associated metadata, and then commit these to the database. Additional metadata will be associated with interactions such as PubMed/PSIMI identifiers, labels and the source databases. Directionality information is retained such that separate rows will be added for the interactions `(A, B)` and `(B, A)` where `A` is the source node and `B` is the target node. This information is particularly useful for analysing Kegg interactions, which have directionality information in the labels.

In [8]:
from src.database.utilities import create_interactions, create_proteins

proteins: List[models.Protein] = create_proteins((
    entry for entry in uniprot_entries if entry is not None
))
logger.info(f"Created {len(proteins)} protein rows")

[INFO] 2019-09-05 12:50:17,922 uniprot Requesting 41 batches of size 500.
100%|██████████| 41/41 [1:00:17<00:00, 88.24s/it]
[INFO] 2019-09-05 13:51:10,170 uniprot Parsing XML into UniprotEntry instances.
[INFO] 2019-09-05 14:05:59,851 utilities Populating annotation tables.
[INFO] 2019-09-05 14:06:33,153 utilities Populating gene symbol table.
[INFO] 2019-09-05 14:06:35,144 utilities Populating uniprot identifier table.
[INFO] 2019-09-05 14:06:39,314 utilities Updating Protein table.
[INFO] 2019-09-05 14:20:58,687 <ipython-input-8-2d88604cf55f> Created 19552 protein rows


In [9]:
interactions: List[models.Interaction] = create_interactions(
    mapped_interactions
)
logger.info(f"Created {len(interactions)} interaction rows")

[INFO] 2019-09-05 14:21:10,954 utilities Populating interaction evidence and metadata tables.
[INFO] 2019-09-05 14:23:51,478 utilities Populating interaction table.
[WARNING] 2019-09-05 14:24:02,166 utilities Source node 'Q5JVB6' was not found in protein table. Skipping interaction {'source': 'Q5JVB6', 'target': 'Q9P1W9', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:16169070', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-05 14:24:02,433 utilities Source node 'P0DPI2' was not found in protein table. Skipping interaction {'source': 'P0DPI2', 'target': 'P41235', 'labels': [], 'databases': ['innatedb'], 'evidence': [{'pubmed': 'PUBMED:14988562', 'psimi': 'MI:0030'}]}.
[WARNING] 2019-09-05 14:24:05,582 utilities Source node 'E9PCH2' was not found in protein table. Skipping interaction {'source': 'E9PCH2', 'target': 'Q5SVZ6', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:24:06,372 utilities Target node 'P0DPI2' was not found in protein 

[WARNING] 2019-09-05 14:25:47,101 utilities Target node 'Q14CA7' was not found in protein table. Skipping interaction {'source': 'P68400', 'target': 'Q14CA7', 'labels': ['phosphorylation'], 'databases': ['hprd'], 'evidence': [{'pubmed': 'PUBMED:16964243', 'psimi': None}, {'pubmed': 'PUBMED:17287340', 'psimi': None}, {'pubmed': 'PUBMED:17693683', 'psimi': None}, {'pubmed': 'PUBMED:17924679', 'psimi': None}, {'pubmed': 'PUBMED:18212344', 'psimi': None}, {'pubmed': 'PUBMED:18452278', 'psimi': None}, {'pubmed': 'PUBMED:18669648', 'psimi': None}, {'pubmed': 'PUBMED:18767875', 'psimi': None}, {'pubmed': 'PUBMED:19413330', 'psimi': None}, {'pubmed': 'PUBMED:19415658', 'psimi': None}, {'pubmed': 'PUBMED:19651622', 'psimi': None}, {'pubmed': 'PUBMED:19664994', 'psimi': None}, {'pubmed': 'PUBMED:19664995', 'psimi': None}, {'pubmed': 'PUBMED:20068231', 'psimi': None}, {'pubmed': 'PUBMED:8002936', 'psimi': None}, {'pubmed': 'PUBMED:8376365', 'psimi': None}]}.
[WARNING] 2019-09-05 14:25:47,205 util

[WARNING] 2019-09-05 14:26:47,886 utilities Target node 'Q5SP12' was not found in protein table. Skipping interaction {'source': 'Q8N7X8', 'target': 'Q5SP12', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:26:49,785 utilities Target node 'P0DPA3' was not found in protein table. Skipping interaction {'source': 'P51854', 'target': 'P0DPA3', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:26:53,545 utilities Source node 'P0DPB3' was not found in protein table. Skipping interaction {'source': 'P0DPB3', 'target': 'Q9Y6D6', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:20195357', 'psimi': 'MI:0034'}]}.
[WARNING] 2019-09-05 14:26:56,284 utilities Target node 'P0DN86' was not found in protein table. Skipping interaction {'source': 'P01225', 'target': 'P0DN86', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:10563789', 'psimi': 'MI:0492'}]}.
[WARNING] 2019-09-05 14:26:59,056 utilitie

[WARNING] 2019-09-05 14:28:39,028 utilities Target node 'A2A284' was not found in protein table. Skipping interaction {'source': 'P12931', 'target': 'A2A284', 'labels': ['phosphorylation'], 'databases': ['hprd'], 'evidence': [{'pubmed': 'PUBMED:16497976', 'psimi': None}, {'pubmed': 'PUBMED:1689310', 'psimi': None}, {'pubmed': 'PUBMED:17016520', 'psimi': None}, {'pubmed': 'PUBMED:17389395', 'psimi': None}, {'pubmed': 'PUBMED:20068231', 'psimi': None}, {'pubmed': 'PUBMED:7682059', 'psimi': None}, {'pubmed': 'PUBMED:8657103', 'psimi': None}, {'pubmed': 'PUBMED:8885868', 'psimi': None}]}.
[WARNING] 2019-09-05 14:28:43,036 utilities Target node 'B1AN89' was not found in protein table. Skipping interaction {'source': 'Q8N7X8', 'target': 'B1AN89', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:28:43,913 utilities Target node 'A6NGZ7' was not found in protein table. Skipping interaction {'source': 'Q9GZM5', 'target': 'A6NGZ7', 'labels': [], 'databases': ['biop

[WARNING] 2019-09-05 14:30:07,191 utilities Source node 'Q5T3U0' was not found in protein table. Skipping interaction {'source': 'Q5T3U0', 'target': 'Q6UY14', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:16189514', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-05 14:30:07,562 utilities Target node 'Q14CA7' was not found in protein table. Skipping interaction {'source': 'P17612', 'target': 'Q14CA7', 'labels': ['phosphorylation'], 'databases': ['hprd'], 'evidence': [{'pubmed': 'PUBMED:16964243', 'psimi': None}, {'pubmed': 'PUBMED:17287340', 'psimi': None}, {'pubmed': 'PUBMED:17693683', 'psimi': None}, {'pubmed': 'PUBMED:17924679', 'psimi': None}, {'pubmed': 'PUBMED:18212344', 'psimi': None}, {'pubmed': 'PUBMED:18452278', 'psimi': None}, {'pubmed': 'PUBMED:18669648', 'psimi': None}, {'pubmed': 'PUBMED:18767875', 'psimi': None}, {'pubmed': 'PUBMED:19413330', 'psimi': None}, {'pubmed': 'PUBMED:19415658', 'psimi': None}, {'pubmed': 'PUBMED:19651622', 'psimi': None}, {'pub

...

[WARNING] 2019-09-05 14:47:18,277 utilities Target node 'P0DPB3' was not found in protein table. Skipping interaction {'source': 'P67775', 'target': 'P0DPB3', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:20195357', 'psimi': 'MI:0034'}]}.
[WARNING] 2019-09-05 14:47:21,416 utilities Source node 'A8MUN9' was not found in protein table. Skipping interaction {'source': 'A8MUN9', 'target': 'P40939', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:47:25,769 utilities Target node 'P0DPA3' was not found in protein table. Skipping interaction {'source': 'P42566', 'target': 'P0DPA3', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:19380743', 'psimi': 'MI:0004'}]}.
[WARNING] 2019-09-05 14:47:25,774 utilities Source node 'F5H792' was not found in protein table. Skipping interaction {'source': 'F5H792', 'target': 'Q96D46', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:47:30,133 utilitie

[WARNING] 2019-09-05 14:48:22,562 utilities Target node 'P0DPB6' was not found in protein table. Skipping interaction {'source': 'Q9NW08', 'target': 'P0DPB6', 'labels': [], 'databases': ['innatedb', 'pina2'], 'evidence': [{'pubmed': 'PUBMED:22939629', 'psimi': 'MI:0401'}]}.
[WARNING] 2019-09-05 14:48:22,808 utilities Source node 'F4EBP3' was not found in protein table. Skipping interaction {'source': 'F4EBP3', 'target': 'P0CG48', 'labels': [], 'databases': ['innatedb'], 'evidence': [{'pubmed': 'PUBMED:22505724', 'psimi': 'MI:0004'}]}.
[WARNING] 2019-09-05 14:48:23,584 utilities Source node 'A2A457' was not found in protein table. Skipping interaction {'source': 'A2A457', 'target': 'P15090', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:48:25,176 utilities Target node 'Q7Z6S6' was not found in protein table. Skipping interaction {'source': 'O43379', 'target': 'Q7Z6S6', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:16169070', 'ps

[WARNING] 2019-09-05 14:49:51,631 utilities Target node 'Q9Y2A0' was not found in protein table. Skipping interaction {'source': 'P21673', 'target': 'Q9Y2A0', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:16169070', 'psimi': 'MI:0007'}, {'pubmed': 'PUBMED:16169070', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-05 14:49:51,968 utilities Target node 'P0DPB6' was not found in protein table. Skipping interaction {'source': 'P46109', 'target': 'P0DPB6', 'labels': [], 'databases': ['innatedb', 'pina2'], 'evidence': [{'pubmed': 'PUBMED:21988832', 'psimi': 'MI:0004'}, {'pubmed': 'PUBMED:21988832', 'psimi': 'MI:0018'}, {'pubmed': 'PUBMED:21988832', 'psimi': 'MI:0397'}]}.
[WARNING] 2019-09-05 14:49:55,234 utilities Source node 'P0DPB3' was not found in protein table. Skipping interaction {'source': 'P0DPB3', 'target': 'Q9UKU6', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:20195357', 'psimi': 'MI:0034'}]}.
[WARNING] 2019-09-05 14:50:03,985 utilities So

[WARNING] 2019-09-05 14:51:14,682 utilities Target node 'Q5SP12' was not found in protein table. Skipping interaction {'source': 'P27930', 'target': 'Q5SP12', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:51:19,599 utilities Source node 'A8MVW5' was not found in protein table. Skipping interaction {'source': 'A8MVW5', 'target': 'Q13705', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:51:19,600 utilities Source node 'A2A457' was not found in protein table. Skipping interaction {'source': 'A2A457', 'target': 'Q07283', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:51:23,475 utilities Source node 'A8MT70' was not found in protein table. Skipping interaction {'source': 'A8MT70', 'target': 'Q7L7V1', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:51:25,489 utilities Source node 'P0DPF5' was not found in protein table. Skipping interaction {'source': 'P0DPF5

[WARNING] 2019-09-05 14:52:29,463 utilities Target node 'P0DPA3' was not found in protein table. Skipping interaction {'source': 'Q9H255', 'target': 'P0DPA3', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:52:33,625 utilities Target node 'P0C5W0' was not found in protein table. Skipping interaction {'source': 'P51398', 'target': 'P0C5W0', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:52:34,895 utilities Source node 'Q5T6W5' was not found in protein table. Skipping interaction {'source': 'Q5T6W5', 'target': 'Q9Y580', 'labels': [], 'databases': ['pina2'], 'evidence': [{'pubmed': 'PUBMED:16189514', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-05 14:52:41,261 utilities Source node 'E7EP37' was not found in protein table. Skipping interaction {'source': 'E7EP37', 'target': 'Q15836', 'labels': [], 'databases': ['bioplex'], 'evidence': []}.
[WARNING] 2019-09-05 14:52:43,826 utilities Target node 'Q5SP12' was not found in protein

In [2]:
identifiers = [("P08670", "Q8TF68"), ('Q9P2W3', 'P48051')]
models.Interaction.filter_by_edge([("P08670", "Q8TF68")]).count()

OperationalError: no such column: t1.index

In [3]:
from src.parsers import types

e = types.InteractionEvidenceData('1', '0001')
es = [x.normalize() for x in list([e])]

In [4]:
e

InteractionEvidenceData(pubmed='PUBMED:1', psimi='MI:0001')